In [6]:
import numpy as np
import pandas as pd
import re
np.random.seed(0)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
dat_dir = '../data/'
file_path = dat_dir + 'data_train.csv'
df = pd.read_csv(file_path)
df.head()

,Id,Prediction
0,r44_c1,4
1,r61_c1,3
2,r67_c1,4
3,r72_c1,3
4,r86_c1,5


In [38]:
r_c = np.array(list(map(lambda x:re.split("[r_c]", x), ratings_df.Id)))
df['User'] = np.array([int(x) for x in r_c[:,1]])
df['Item'] = np.array([int(x) for x in r_c[:,3]])
df.head()

,Id,Prediction,User,Item
0,r44_c1,4,44,1
1,r61_c1,3,61,1
2,r67_c1,4,67,1
3,r72_c1,3,72,1
4,r86_c1,5,86,1


In [39]:
df.to_csv(dat_dir + 'data_train_processed.csv')

In [41]:
n_users = df.User.unique().shape[0]
n_items = df.Item.unique().shape[0]
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[3]-1, row[4]-1] = row[2]
ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 5., 3., 3.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 3.]])

In [43]:
np.save('ratings.npy', ratings)

In [45]:
print(str(n_users) + ' users')
print(str(n_items) + ' items')
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

10000 users
1000 items
Sparsity: 11.77%


In [64]:
from pre_post_process import split_data, load_data
_, ratings = load_data(dat_dir + "data_train.csv")
sample_ids, _ = load_data(dat_dir + "sample_submission.csv")

number of items: 1000, number of users: 10000
number of items: 1000, number of users: 10000


In [65]:
valid_ratings, train, test = split_data(ratings, p_test=0.1)

the shape of original ratings. (# of row, # of col): (1000, 10000)
the shape of valid ratings. (# of row, # of col): (1000, 10000)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:1068598
Total number of nonzero elements in test data:108354


In [78]:
train[train.nonzero()].flatten()

AttributeError: flatten not found

In [ ]:
from ExplicitMF import ExplicitMF
# train = train.todense()
# test = test.todense()
MF_ALS = ExplicitMF(train, n_factors=40, \
                    user_fact_reg=0.1, item_fact_reg=0.1, verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 100]
MF_ALS.calculate_learning_curve(iter_array, test)

Iteration: 1


/Users/luke/Documents/Projects/ML/project2/MRS/src/ExplicitMF.py:157: RuntimeWarning: overflow encountered in multiply
  (e * self.user_vecs[u, :] - \
/Users/luke/Documents/Projects/ML/project2/MRS/src/ExplicitMF.py:154: RuntimeWarning: overflow encountered in multiply
  (e * self.item_vecs[i, :] - \
/Users/luke/Documents/Projects/ML/project2/MRS/src/ExplicitMF.py:155: RuntimeWarning: invalid value encountered in multiply
  self.user_fact_reg * self.user_vecs[u,:])
/Users/luke/Documents/Projects/ML/project2/MRS/src/ExplicitMF.py:158: RuntimeWarning: invalid value encountered in multiply
  self.item_fact_reg * self.item_vecs[i,:])
